In [1]:
using Printf
using Random
using LinearAlgebra
using Eirene
using MAT
using Statistics
using Plots
using Glob

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10929387716488947544\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17586774482889706773\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10929387716488947544\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/sppatankar/.julia/packages/Plots/oZheM/src/backends/hdf5.jl", 162, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010fb6611f, Ptr{Nothing} @0x000000010fbffeb3, Ptr{Nothing} @0x000000010fc00fab, Ptr{Nothing} @0x000000010fbff8df, Ptr{Nothing} @0x000000010fbffbbc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010fc191ce, Ptr{Nothing} @0x000000010fc181b1, Ptr{Nothing} @0x000000010fc189d1, Ptr{Nothing} @0x000000010fc190d6, Ptr{Nothing} @0x000000010fbf1b6d, Ptr{Nothing} @0x000000010fc1a27c, Ptr{Nothing} @0x000000011c2ddd4f, Ptr{Nothing} @0x000000013f9da4ee, Ptr{Nothing} @0x000000010fc00fcf, Ptr{Nothing} @0x000000010fbff8df, Ptr{Nothing} @0x000000010fbffbbc, Base.InterpreterIP in top-level CodeInfo for Plots at statement 10, Ptr{Nothing} @0x000000010fc191ce, Ptr{Nothing} @0x000000010fc1a074, Ptr{Nothing} @0x000

In [2]:
function constant_probability(n, p)
    G = zeros(n, n)
    for i = 1:n
        for j = 1:i-1
            r = rand(1)[1]
            if r < p
                G[i, j] = 1
                G[j, i] = 1
            end
        end
    end
    node_order = 1:n
    return G, node_order
end  

function make_weighted_from_order(G, node_order)
    # adapted from code by ASB
    # original at https://github.com/BassettLab/Reorderability_scripts
    reordered_G = G[node_order, node_order] # often unnecessary
    n = length(node_order) # number of nodes
    val_mat = ones(n, n)
    for col = 1:n
        val_mat[1:col, col] = val_mat[1:col, col] * col
        val_mat[col, 1:col] = val_mat[col, 1:col] * col
    end
    weighted_G = reordered_G .* val_mat
    # replace 0 weighted edges with the largest edge weight possible
    # this is equivalent to assigning these edges the worst rank possible
    replace!(weighted_G, 0 => 2 * n)  
    # weighted_G[findall(A -> A .== 0, weighted_G)] .= 2 * n would work too
    weighted_G[diagind(weighted_G)] .= 0 # set diagonal to 0
    return weighted_G # edges here are ranked by order of appearance
end

function bettiCurveFromBarcode(barcode_array,nNodes,nmats,maxDim)
    nNodes = Int(nNodes)
    nmats = Int(nmats)
    maxDim = Int(maxDim)
    bettiBar = zeros(nmats,maxDim)
    bettiCurve = zeros(nmats,nNodes+1,maxDim)
    birthCurve = zeros(nmats,nNodes,maxDim)
    deathCurve = zeros(nmats,nNodes,maxDim)
    for dimn in collect(1:maxDim)
        dimn = Int(dimn)
        for matn in collect(1:nmats)
            matn = Int(matn)
            bb = 0
            currentCurve = barcode_array[matn,:]
            currentCurveDim = currentCurve[dimn]
            for barn in collect(1:size(currentCurveDim,1))
                # Add to birth curve
                birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] = birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] .+1
                if currentCurveDim[barn,2]>nNodes
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] .+1
                    bb = bb+(nNodes+1-currentCurveDim[barn,1])
                else
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn].+1
                    deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] = deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] .+1
                    bb = bb+(currentCurveDim[barn,2] - currentCurveDim[barn,1])
                end
            end
            bettiBar[matn,dimn] = deepcopy(bb)
        end
    end
    return bettiCurve, birthCurve, deathCurve, bettiBar
end

function plotBarcode(allPIs,nNodes,graphN,maxDim,fontSize)
    nNodes = Int(nNodes)
    graphn = Int(graphN)
    maxDim = Int(maxDim)
    counter1 = 0
    pbar = plot(1:6,zeros(6),c=:black)
    colors = [:blue :green :red]
    for dim in collect(1:maxDim)
        barn = barcode_array[graphN, dim]
        barn = barn[sortperm(barn[:,1]),:]
        nbars = size(barn)[1]
        for cntr1 in collect(1:nbars)
            birth = barn[cntr1,1]
            death = barn[cntr1,2]
            plot!([birth, death],[cntr1+counter1, cntr1+counter1],c=colors[dim], legend = false,
                            xlim = (0,nNodes), ytickfont = font(fontSize), xtickfont = font(fontSize))
        end
        display(pbar)
        counter1 = counter1+nbars
    end
    return pbar
end

plotBarcode (generic function with 1 method)

In [3]:
pwd()

"/Volumes/My Passport/Curiosity/v2"

In [4]:
data_in_path = "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene"
data_out_path = "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_processed_Eirene"
all_topics = glob("*.mat", data_in_path)

40-element Array{String,1}:
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/Boolean_algebra.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/abstract_algebra.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/accounting.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/anatomy.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/biochemistry.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/biophysics.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/calculus.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/chemistry.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/cognitive_science.mat"
 "/U

In [8]:
topic = all_topics[6]

"/Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/biophysics.mat"

In [11]:
n_dims = 3 # number of dimensions to track persistent homology in

i = 6;
@printf("Topic: %s\n", all_topics[i])
all_vars = matread(all_topics[i])
nodes = all_vars["nodes"]
n = length(nodes)
adj = Array(all_vars["adj"])
edge_info = all_vars["edge_info"]
topic_ID = all_vars["topic"]
weighted_G = make_weighted_from_order(adj, 1:n)

Topic: /Users/sppatankar/Developer/My Passport/Curiosity/v2/Data/Wiki/Wiki_preprocessed_Eirene/biophysics.mat


463×463 Array{Float64,2}:
   0.0  926.0  926.0  926.0  926.0  …  926.0  926.0  926.0  926.0  926.0
 926.0    0.0  926.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0    0.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0    0.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0    0.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0    6.0  …  926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0  926.0     459.0  460.0  461.0  462.0  926.0
 926.0  926.0  926.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0  926.0  …  926.0  926.0  926.0  926.0  926.0
 926.0  926.0  926.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
 926.0  926.0   13.0  926.0  926.0     926.0  926.0  926.0  926.0  926.0
   ⋮                     

In [16]:
save_string = string(data_out_path, "/", "weight_G_", topic_ID, ".mat")
matwrite(save_string, 
    Dict("n" => n,
        "adj" => adj,
        "weighted_adj" => weighted_G))

In [ ]:
barcode_array = Array{Array{Float64}}(undef, 3) # iters = 1 implied
betti_curves = Array{Array{Float64}}(undef, 3) # iters = 1 implied
C = Eirene.eirene(weighted_G, model = "vr", maxdim = n_dims, record = "none")


In [ ]:
for k in 1:n_dims
    barcode_array[k] = barcode(C, dim = k)
    betti_curves[k] = betticurve(C, dim = k)
end
save_string = string(data_out_path, "/", topic, ".mat")
matwrite(save_string, 
    Dict("n" => n,
        "adj" => adj,
        "weighted_adj" => weighted_G,
        "barcode_array" => barcode_array,
        "betti_curves" => betti_curves))